In [1]:
pip install PyMuPDF

     |████████████████████████████████| 8.8 MB 8.5 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
import re
import json
import pickle
import os
import sys,fitz

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def clean_text(resume_text):
    stopwords_set = set(stopwords.words('english')+['``',"''"])
    resume_text = re.sub('http\S+\s*', ' ', resume_text)  # remove URLs
    resume_text = re.sub('RT|cc', ' ', resume_text)  # remove RT and cc
    resume_text = re.sub('#\S+', '', resume_text)  # remove hashtags
    resume_text = re.sub('@\S+', '  ', resume_text)  # remove mentions
    resume_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resume_text)  # remove punctuations
    resume_text = re.sub(r'[^\x00-\x7f]',r' ', resume_text) 
    resume_text = re.sub('\s+', ' ', resume_text)  # remove extra whitespace
    resume_text = resume_text.lower()  # convert to lowercase
    resume_text_tokens = word_tokenize(resume_text)  # tokenize
    filtered_text = [w for w in resume_text_tokens if not w in stopwords_set]  # remove stopwords
    return ' '.join(filtered_text)

In [7]:
def get_title(text):
  with open('/content/gdrive/MyDrive/LSTM MODEL/feature_tokenizer.pickle', 'rb') as handle:
    feature_tokenizer = pickle.load(handle)         
         
  with open('/content/gdrive/MyDrive/LSTM MODEL/dictionary.pickle', 'rb') as handle:
      encoding_to_label = pickle.load(handle)

  with open("/content/gdrive/MyDrive/LSTM MODEL/labels.json", "r") as read_file:
              original_labels = json.load(read_file)
  
  

  sen=clean_text(text)
  text=sen
  

  max_length = 500
  trunc_type = 'post'
  padding_type = 'post'

  predict_sequences = feature_tokenizer.texts_to_sequences([text])
  predict_padded = pad_sequences(predict_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  predict_padded = np.array(predict_padded)

  model = keras.models.load_model('/content/gdrive/MyDrive/LSTM MODEL/LSTM_model')
  prediction = model.predict(predict_padded)

  encodings = np.argpartition(prediction[0], -8)[-8:]
  encodings = encodings[np.argsort(prediction[0][encodings])]
  encodings = reversed(encodings)

  data = {}

  for encoding in encodings:
    label = encoding_to_label[encoding]
    probability = prediction[0][encoding] * 100
    probability = round(probability, 2)
    data[original_labels[label]]=probability

  if '.NET Developer' in data.keys():
    del(data['.NET Developer'])

  print(data)

  titles=[[],[]]
  for key,values in data.items():
    titles[0].append(key)
    titles[1].append(values)
  
  new_data={}
  new_data[titles[0][0]]=titles[1][0]
  new_data[titles[0][1]]=titles[1][1]

  return new_data





In [9]:
with open("/content/Zohaib Khan - CV for Research Assistant - Lab Instructor.txt",'r',encoding='utf-8',errors='ignore') as f1:
  sentence2=f1.read()

test=get_title(sentence2)
print(test)



{'Data Scientist': 24.13, 'Automation Tester': 20.36, 'Advocate': 11.46, 'Health and Fitness': 8.27, 'SAP Developer': 7.7, 'Mechanical Engineer': 7.18, 'Business Analyst': 5.35, 'Blockchain Developer': 3.0}
{'Data Scientist': 24.13, 'Automation Tester': 20.36}
